In [2]:
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from src.config import NEO4J_URI, NEO4J_AUTH

# 1. Setup Connection (imported from config)
URI = NEO4J_URI
AUTH = NEO4J_AUTH

# 2. Define the SAME embedding model you used to create the index
# (It must match exactly so the math aligns)
embedding_model = OpenAIEmbeddings(
    model="text-embedding-granite-embedding-107m-multilingual",  # Updated to match setup - supports multilingual!
    openai_api_base="http://127.0.0.1:1234/v1",   # Fixed URL to match setup
    openai_api_key="",
    check_embedding_ctx_length=False
)

def search_diagnosis(query):
    print(f"🔎 Searching for: '{query}'...")

    # 3. Load the existing index
    # Note: We use 'from_existing_graph' to connect to the existing vector index
    vector_store = Neo4jVector.from_existing_graph(
        embedding=embedding_model,
        url=URI,
        username=AUTH[0],
        password=AUTH[1],
        index_name="diagnosis_rules_index",  # Must match the name you created earlier
        node_label="Diagnosis",                     # FIXED: Added required parameter
        text_node_properties=["name", "code"],
        embedding_node_property="embedding",        # FIXED: Added required parameter
    )

    # 4. Perform Vector Search
    # k=3 means "give me the top 3 closest matches"
    results = vector_store.similarity_search_with_score(query, k=1)

    # 5. Print Results
    print("\n✅ Results found:")
    for doc, score in results:
        # Neo4jVector returns the properties you indexed in 'page_content' 
        # or within doc.metadata depending on configuration.
        print(f"------------------------------------------------")
        print(f"Score: {score:.4f}")  # Closer to 1.0 is better
        print(f"Content: {doc.page_content}") # This usually contains the 'name' and 'code'
        print(f"Metadata: {doc.metadata}")

if __name__ == "__main__":
    # Test both English and Indonesian queries
    print("=== Testing English query ===")
    search_diagnosis("heart attack")
    
    print("\n=== Testing Indonesian query ===")
    search_diagnosis("serangan jantung")

=== Testing English query ===
🔎 Searching for: 'heart attack'...

✅ Results found:
------------------------------------------------
Score: 0.8684
Content: 
name: Acute Myocardial Infarction (Heart Attack)
code: I21.9
Metadata: {'severity': 'High', 'avg_cost': 65000000.0}

=== Testing Indonesian query ===
🔎 Searching for: 'serangan jantung'...

✅ Results found:
------------------------------------------------
Score: 0.8442
Content: 
name: Acute Myocardial Infarction (Heart Attack)
code: I21.9
Metadata: {'severity': 'High', 'avg_cost': 65000000.0}
